<a href="https://colab.research.google.com/github/Tubby101/Machine_learning_projects/blob/main/Oddesy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('odyssey.mb.txt', 'http://classics.mit.edu/Homer/odyssey.mb.txt')

  16384/Unknown - 0s 1us/step

In [3]:

# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')


Length of text: 611300 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

Provided by The Internet Classics Archive.
See bottom for copyright. Available online at
    http://classics.mit.edu//Homer/odyssey.html

The Odyssey
By Homer


Translated by Samuel Butler

------------------------------------------------------------
74 unique characters


In [5]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [6]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [7]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[49, 50, 51, 52, 53, 54, 55], [72, 73, 74]]>

In [8]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [9]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [10]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [11]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [12]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(611300,), dtype=int64, numpy=array([36, 66, 63, ..., 27, 10,  1])>

In [13]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [14]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

P
r
o
v
i
d
e
d
 
b


In [15]:
seq_length = 100

In [16]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'P' b'r' b'o' b'v' b'i' b'd' b'e' b'd' b' ' b'b' b'y' b' ' b'T' b'h'
 b'e' b' ' b'I' b'n' b't' b'e' b'r' b'n' b'e' b't' b' ' b'C' b'l' b'a'
 b's' b's' b'i' b'c' b's' b' ' b'A' b'r' b'c' b'h' b'i' b'v' b'e' b'.'
 b'\n' b'S' b'e' b'e' b' ' b'b' b'o' b't' b't' b'o' b'm' b' ' b'f' b'o'
 b'r' b' ' b'c' b'o' b'p' b'y' b'r' b'i' b'g' b'h' b't' b'.' b' ' b'A'
 b'v' b'a' b'i' b'l' b'a' b'b' b'l' b'e' b' ' b'o' b'n' b'l' b'i' b'n'
 b'e' b' ' b'a' b't' b'\n' b' ' b' ' b' ' b' ' b'h' b't' b't' b'p' b':'
 b'/' b'/' b'c'], shape=(101,), dtype=string)


In [17]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'Provided by The Internet Classics Archive.\nSee bottom for copyright. Available online at\n    http://c'
b'lassics.mit.edu//Homer/odyssey.html\n\nThe Odyssey\nBy Homer\n\n\nTranslated by Samuel Butler\n\n------------'
b'----------------------------------------------------------\n\nBOOK I\n\nTell me, O muse, of that ingeniou'
b's hero who travelled far and wide\nafter he had sacked the famous town of Troy. Many cities did he vis'
b'it,\nand many were the nations with whose manners and customs he was acquainted;\nmoreover he suffered '


In [18]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [19]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [20]:
dataset = sequences.map(split_input_target)

In [21]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Provided by The Internet Classics Archive.\nSee bottom for copyright. Available online at\n    http://'
Target: b'rovided by The Internet Classics Archive.\nSee bottom for copyright. Available online at\n    http://c'


In [22]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [23]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [24]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [25]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [26]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 75) # (batch_size, sequence_length, vocab_size)


In [27]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  19200     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  76875     
                                                                 
Total params: 4,034,379
Trainable params: 4,034,379
Non-trainable params: 0
_________________________________________________________________


In [28]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [29]:
sampled_indices

array([50,  4,  5, 19, 28, 42, 26, 65, 56, 32,  7, 40, 70, 59, 23, 57,  5,
       21, 59,  5, 29, 11, 14, 57,  3, 39,  8, 59, 71, 72, 48, 24, 31, 33,
       26, 15, 41, 18, 38, 13, 37, 48, 42, 42, 52, 36, 37, 25, 74, 14, 50,
       36, 56, 19, 46, 25, 15, 11,  7, 44, 50, 14, 20, 26, 41, 25, 38, 43,
       27, 21, 18, 60, 55, 20,  9, 47, 28,  6, 38, 36, 72, 13, 65, 27, 27,
       36, 43, 59, 31, 36, 73, 62, 36, 12,  3, 27, 72,  9, 46, 11])

In [30]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'd heavy upon me; but as regards your question, there is an island\nfar away in the sea which is calle'

Next Char Predictions:
 b'b"\'?HVFqhL)TvkCi\'Ak\'I/2i!S,kwx]DKMF4U;R1Q]VVdPQEz2bPh?ZE4/)Xb2@FUERWGA;lg@-[H(RPx1qGGPWkKPynP0!Gx-Z/'


In [31]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [32]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 75)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.3170867, shape=(), dtype=float32)


In [33]:
tf.exp(example_batch_mean_loss).numpy()

74.9699

In [34]:
model.compile(optimizer='adam', loss=loss)

In [35]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [36]:
EPOCHS = 30

In [37]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
94/94 [==============================] - 10s 52ms/step - loss: 2.8985
Epoch 2/30
94/94 [==============================] - 5s 48ms/step - loss: 2.1657
Epoch 3/30
94/94 [==============================] - 5s 48ms/step - loss: 1.9187
Epoch 4/30
94/94 [==============================] - 5s 49ms/step - loss: 1.7128
Epoch 5/30
94/94 [==============================] - 5s 50ms/step - loss: 1.5563
Epoch 6/30
94/94 [==============================] - 5s 49ms/step - loss: 1.4426
Epoch 7/30
94/94 [==============================] - 5s 50ms/step - loss: 1.3561
Epoch 8/30
94/94 [==============================] - 6s 50ms/step - loss: 1.2889
Epoch 9/30
94/94 [==============================] - 5s 50ms/step - loss: 1.2303
Epoch 10/30
94/94 [==============================] - 5s 50ms/step - loss: 1.1787
Epoch 11/30
94/94 [==============================] - 5s 50ms/step - loss: 1.1287
Epoch 12/30
94/94 [==============================] - 5s 50ms/step - loss: 1.0817
Epoch 13/30
94/94 [=================

In [38]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [39]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [40]:
start = time.time()
states = None
next_char = tf.constant(['Hello'])
result = [next_char]

for n in range(50):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Hellous
shut up small and shote the wine, and dreved th 

________________________________________________________________________________

Run time: 0.908409595489502


In [41]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [42]:
states = None
next_char = tf.constant(['You suck '])
result = [next_char]

for n in range(500):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

You suck to him in my own house
and come home, and see if you can tell me what I think will be all his displeaser,
and I will tell you everything that is going to happen over the
waters of Oceanus the sons of Arcenifemens,
but let us see whether it is long about the aim." 

Ulysses answered, "madam, or I have been long about the country or staying in my own house
to persuase the suitors in your house, whichever you please, but I used two sons

Hadome down to the sea shore, webe, stoo, at which the
work o
